### Inactivated Forms & Loglines – Intermediate Aggregate

This notebook produces a subject-level operational aggregate summarizing
inactivated forms and records across studies and sites.

Role in pipeline:
- Stored as an intermediate artifact
- Intended for Phase 3 (data quality metrics)
- Intended for Phase 4 (operational insights)
- Not merged into the Phase 2 master baseline table


In [1]:
# imports
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


In [2]:
# Define data paths
DATA_DIR = Path("../data")
INTERMEDIATE_DIR = DATA_DIR / "intermediate"

inventory = pd.read_csv(INTERMEDIATE_DIR / "file_inventory.csv")

inactivated_forms = inventory[inventory.file_type == "other"]
inactivated_forms.shape

(49, 3)

In [3]:
# Sanity check: display first few rows
inactivated_forms["file_name"].head()

0             Study 5_Global Missing Pages_updated.xlsx
4     Study 5_Inactivated Forms and Folders_updated....
6                    Study 5_Compiled EDRR_updated.xlsx
9             Global Coding Report_Medra 4_updated.xlsx
12    Study 21_Visit_Projection_Tracker_1_30OCT2025_...
Name: file_name, dtype: object

In [4]:
dfs = []

for _, row in inactivated_forms.iterrows():
    path = Path(row["file_path"])
    try:
        df = pd.read_excel(path)
        df["source_file"] = path.name
        dfs.append(df)
    except Exception as e:
        print(f"Failed to read {path.name}: {e}")

len(dfs)


49

In [5]:
inactivated_forms_raw = pd.concat(dfs, ignore_index=True)
inactivated_forms_raw.shape

/tmp/ipykernel_48918/2490196848.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  inactivated_forms_raw = pd.concat(dfs, ignore_index=True)


(555988, 90)

In [6]:
# Normalize column names
def normalize_colnames(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(r"[^a-z0-9]+", "_", regex=True)
        .str.replace(r"_+", "_", regex=True)
        .str.strip("_")
    )
    return df

inactivated = normalize_colnames(inactivated_forms_raw)
inactivated.columns.tolist()

['study_name',
 'sitegroupname_countryname',
 'sitenumber',
 'subjectname',
 'overall_subject_status',
 'form_1_subject_status',
 'foldername',
 'visit_date',
 'form_type_summary_or_visit',
 'no_days_page_missing',
 'source_file',
 'country',
 'study_site_number',
 'subject',
 'folder',
 'form',
 'data_on_form_record',
 'recordpostion',
 'audit_action',
 'study',
 'total_open_issue_count_per_subject',
 '',
 'meddra_coding_report',
 'dictionary',
 'dictionary_version_number',
 'form_oid',
 'logline',
 'field_oid',
 'coding_status',
 'require_coding',
 'site',
 'visit',
 'actual_date',
 'days_outstanding',
 'site_number',
 'form_name',
 'lab_category',
 'lab_date',
 'test_name',
 'test_description',
 'issue',
 'comments',
 'form',
 'recordposition',
 'project_name',
 'region',
 'site_id',
 'subject_id',
 'latest_visit_sv_source_rave_edc_bo4',
 'subject_status_source_primary_form',
 'input_files',
 'unnamed_8',
 'unnamed_9',
 'unnamed_10',
 'unnamed_11',
 'unnamed_12',
 'unnamed_13',
 'un

In [10]:
# Cleaning data

# Removing unnamed columns
unnamed_cols = [c for c in inactivated.columns if c.startswith("unnamed")]
print(len(unnamed_cols))

# Removing ' ' columns
space_cols = [c for c in inactivated.columns if c.strip() == '']
print(len(space_cols))

0
1


In [11]:
# Dropping unnamed columns (ignore any that aren't present)
inactivated.drop(columns=unnamed_cols, errors="ignore", inplace=True)

# Dropping space columns (ignore any that aren't present)
inactivated.drop(columns=space_cols, errors="ignore", inplace=True)

In [12]:
# Sanity check: Checking columns after dropping
inactivated.columns.tolist()

['study_name',
 'sitegroupname_countryname',
 'sitenumber',
 'subjectname',
 'overall_subject_status',
 'form_1_subject_status',
 'foldername',
 'visit_date',
 'form_type_summary_or_visit',
 'no_days_page_missing',
 'source_file',
 'country',
 'study_site_number',
 'subject',
 'folder',
 'form',
 'data_on_form_record',
 'recordpostion',
 'audit_action',
 'study',
 'total_open_issue_count_per_subject',
 'meddra_coding_report',
 'dictionary',
 'dictionary_version_number',
 'form_oid',
 'logline',
 'field_oid',
 'coding_status',
 'require_coding',
 'site',
 'visit',
 'actual_date',
 'days_outstanding',
 'site_number',
 'form_name',
 'lab_category',
 'lab_date',
 'test_name',
 'test_description',
 'issue',
 'comments',
 'form',
 'recordposition',
 'project_name',
 'region',
 'site_id',
 'subject_id',
 'latest_visit_sv_source_rave_edc_bo4',
 'subject_status_source_primary_form',
 'input_files',
 'cpmd',
 'ssm',
 'visit_level_subject_status',
 'formname',
 'projected_date']

In [13]:
rename_map = {
    # Study / Project
    "study_name": "study_id",
    "study": "study_id",
    "project_name": "project_name",

    # Geography / Site
    "region": "region",
    "sitegroupname_countryname": "country",
    "country": "country",
    "site": "site_id",
    "site_id": "site_id",
    "sitenumber": "site_id",
    "site_number": "site_id",
    "study_site_number": "site_id",

    # Subject
    "subjectname": "subject_id",
    "subject": "subject_id",
    "subject_id": "subject_id",

    # Subject Status (keep levels distinct)
    "overall_subject_status": "subject_status_overall",
    "visit_level_subject_status": "subject_status_visit",
    "form_1_subject_status": "subject_status_form",
    "subject_status_source_primary_form": "subject_status_primary",

    # Visit / Dates
    "visit": "visit_name",
    "latest_visit_sv_source_rave_edc_bo4": "latest_visit",
    "visit_date": "visit_date",
    "actual_date": "actual_date",
    "projected_date": "projected_date",
    "days_outstanding": "days_outstanding",

    # Folder / Form / Record
    "foldername": "folder_name",
    "folder": "folder_name",
    "form": "form_name",
    "formname": "form_name",
    "form_name": "form_name",
    "form_type_summary_or_visit": "form_type",
    "data_on_form_record": "data_on_record",
    "recordpostion": "record_position",
    "recordposition": "record_position",
    "audit_action": "audit_action",

    # Missingness / Delays
    "no_days_page_missing": "days_page_missing",

    # Coding / Dictionary (logline-level)
    "meddra_coding_report": "coding_system",
    "dictionary": "dictionary_name",
    "dictionary_version_number": "dictionary_version",
    "form_oid": "form_oid",
    "logline": "logline_id",
    "field_oid": "field_oid",
    "coding_status": "coding_status",
    "require_coding": "require_coding",

    # Lab-related (cross-source bleed, keep for assessment)
    "lab_category": "lab_category",
    "lab_date": "lab_date",
    "test_name": "lab_test_name",
    "test_description": "lab_test_description",
    "issue": "lab_issue",
    "comments": "comments",

    # Metrics bleeding from CPID / EDRR (important to detect redundancy)
    "total_open_issue_count_per_subject": "open_issue_count_subject",
    "input_files": "input_files",
    "cpmd": "cpmd",
    "ssm": "ssm",

    # Metadata
    "source_file": "source_file"
}


In [14]:
# Apply renaming
inactivated.rename(columns=rename_map, inplace=True)

In [15]:
# Checking final columns
inactivated.columns.tolist()

['study_id',
 'country',
 'site_id',
 'subject_id',
 'subject_status_overall',
 'subject_status_form',
 'folder_name',
 'visit_date',
 'form_type',
 'days_page_missing',
 'source_file',
 'country',
 'site_id',
 'subject_id',
 'folder_name',
 'form_name',
 'data_on_record',
 'record_position',
 'audit_action',
 'study_id',
 'open_issue_count_subject',
 'coding_system',
 'dictionary_name',
 'dictionary_version',
 'form_oid',
 'logline_id',
 'field_oid',
 'coding_status',
 'require_coding',
 'site_id',
 'visit_name',
 'actual_date',
 'days_outstanding',
 'site_id',
 'form_name',
 'lab_category',
 'lab_date',
 'lab_test_name',
 'lab_test_description',
 'lab_issue',
 'comments',
 'form_name',
 'record_position',
 'project_name',
 'region',
 'site_id',
 'subject_id',
 'latest_visit',
 'subject_status_primary',
 'input_files',
 'cpmd',
 'ssm',
 'subject_status_visit',
 'form_name',
 'projected_date']

In [16]:
inactivated.head()

,study_id,country,site_id,subject_id,subject_status_overall,subject_status_form,folder_name,visit_date,form_type,days_page_missing,source_file,country,site_id,subject_id,folder_name,form_name,data_on_record,record_position,audit_action,study_id,open_issue_count_subject,coding_system,dictionary_name,dictionary_version,form_oid,logline_id,field_oid,coding_status,require_coding,site_id,visit_name,actual_date,days_outstanding,site_id,form_name,lab_category,lab_date,lab_test_name,lab_test_description,lab_issue,comments,form_name,record_position,project_name,region,site_id,subject_id,latest_visit,subject_status_primary,input_files,cpmd,ssm,subject_status_visit,form_name,projected_date
0,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
inactivated.describe(include="all")

,study_id,country,site_id,subject_id,subject_status_overall,subject_status_form,folder_name,visit_date,form_type,days_page_missing,source_file,country,site_id,subject_id,folder_name,form_name,data_on_record,record_position,audit_action,study_id,open_issue_count_subject,coding_system,dictionary_name,dictionary_version,form_oid,logline_id,field_oid,coding_status,require_coding,site_id,visit_name,actual_date,days_outstanding,site_id,form_name,lab_category,lab_date,lab_test_name,lab_test_description,lab_issue,comments,form_name,record_position,project_name,region,site_id,subject_id,latest_visit,subject_status_primary,input_files,cpmd,ssm,subject_status_visit,form_name,projected_date
count,3369,3369,3369,3369,3365,228,3369,2623,3369,526.000000,555988,120637,66712,500013,66829,22882,66829,11693.000000,66829,431845,310.000000,431535,431535,431418.000000,431535,431535.000000,431535,431535,431535,1086,1339,354,1086.000000,253,253,253,253,253,253,253,253,43947,38336.000000,52536,52516,52536,52536,52531,52529,5,52546.0,52551.0,340,2441,732
unique,4,43,547,1291,8,6,70,150,4,NaN,39,113,728,24358,846,148,2,NaN,8,6,NaN,1,2,NaN,6,NaN,3,2,2,506,26,NaN,NaN,12,10,3,41,46,46,2,2,601,NaN,6,3,1804,45730,180,8,1,71.0,378.0,5,66,136
top,Study 22,USA,Site 15,Subject 2649,On Trial,Subject continuing,Visit 5 (Month 21),Missing visit date,Visit Level,NaN,Global Coding Report_Medra 4_updated.xlsx,USA,Site 18,Subject 66,Screening,Form 14,Y,NaN,Record Inactivated.,Study 21,NaN,MedDRA Coding Report,MedDRAMedHistory,NaN,MHR002_1,NaN,MHTERM,Coded Term,No,Site 1640,Visit 5 (Month 21),NaN,NaN,Site 177,Form 1,CHEMISTRY,28-OCT-2025,APTT,Activated partial\nthromboplastin time,Missing Lab name,Action for Site,From 5,NaN,Study 21,EMEA,Site 3,Subject 7,Screening (1),Screen Failure,Missing Visits,1.0,8.0,Subject continuing,Form 2,05NOV2025
freq,2202,984,56,31,2420,113,1073,2097,2094,NaN,431523,18856,5589,688,12641,5065,43977,NaN,17188,431523,NaN,431535,422104,NaN,310494,NaN,396871,431418,431418,27,391,NaN,NaN,83,87,127,46,15,15,190,190,2964,NaN,26442,25359,335,4,8871,18548,5,18046.0,5645.0,286,866,49
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.583650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.363465,NaN,NaN,1.067742,NaN,NaN,27.999971,NaN,7.943692,NaN,NaN,NaN,NaN,NaN,2025-10-15 11:11:11.186440960,30.704420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.640703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,27.100000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,2025-04-11 00:00:00,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,1.000000,NaN,NaN,28.000000,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,2025-10-06 06:00:00,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,1.000000,NaN,NaN,28.000000,NaN,7.000000,NaN,NaN,NaN,NaN,NaN,2025-10-23 00:00:00,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,NaN,NaN,1.000000,NaN,NaN,28.000000,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,2025-11-02 18:00:00,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,28.100000,NaN,198.000000,NaN,NaN,NaN,NaN,NaN,2025-11-06 00:00:00,554.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def coalesce_duplicate_columns(df):
    df = df.copy()
    for col in df.columns[df.columns.duplicated()].unique():
        # select all columns with this name
        dup_cols = df.loc[:, df.columns == col]
        # coalesce row-wise: first non-null value
        df[col] = dup_cols.bfill(axis=1).iloc[:, 0]
        # drop all but one
        df = df.loc[:, ~df.columns.duplicated()]
    return df
inactivated = coalesce_duplicate_columns(inactivated)

In [19]:
inactivated.shape

(555988, 42)

In [20]:
inactivated.head()

,study_id,country,site_id,subject_id,subject_status_overall,subject_status_form,folder_name,visit_date,form_type,days_page_missing,source_file,form_name,data_on_record,record_position,audit_action,open_issue_count_subject,coding_system,dictionary_name,dictionary_version,form_oid,logline_id,field_oid,coding_status,require_coding,visit_name,actual_date,days_outstanding,lab_category,lab_date,lab_test_name,lab_test_description,lab_issue,comments,project_name,region,latest_visit,subject_status_primary,input_files,cpmd,ssm,subject_status_visit,projected_date
0,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Study 5,ARG,Site 165,Subject 2590,On Trial,Subject continuing,Screening,15 SEP 2025,Form 1,60.0,Study 5_Global Missing Pages_updated.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
inactivated.describe(include="all")

,study_id,country,site_id,subject_id,subject_status_overall,subject_status_form,folder_name,visit_date,form_type,days_page_missing,source_file,form_name,data_on_record,record_position,audit_action,open_issue_count_subject,coding_system,dictionary_name,dictionary_version,form_oid,logline_id,field_oid,coding_status,require_coding,visit_name,actual_date,days_outstanding,lab_category,lab_date,lab_test_name,lab_test_description,lab_issue,comments,project_name,region,latest_visit,subject_status_primary,input_files,cpmd,ssm,subject_status_visit,projected_date
count,3369,124006,3369,3369,3365,228,3369,2623,3369,526.000000,555988,22882,66829,11693.000000,66829,310.000000,431535,431535,431418.000000,431535,431535.000000,431535,431535,431535,1339,354,1086.000000,253,253,253,253,253,253,52536,52516,52531,52529,5,52546.0,52551.0,340,732
unique,4,113,547,1291,8,6,70,150,4,NaN,39,148,2,NaN,8,NaN,1,2,NaN,6,NaN,3,2,2,26,NaN,NaN,3,41,46,46,2,2,6,3,180,8,1,71.0,378.0,5,136
top,Study 22,USA,Site 15,Subject 2649,On Trial,Subject continuing,Visit 5 (Month 21),Missing visit date,Visit Level,NaN,Global Coding Report_Medra 4_updated.xlsx,Form 14,Y,NaN,Record Inactivated.,NaN,MedDRA Coding Report,MedDRAMedHistory,NaN,MHR002_1,NaN,MHTERM,Coded Term,No,Visit 5 (Month 21),NaN,NaN,CHEMISTRY,28-OCT-2025,APTT,Activated partial\nthromboplastin time,Missing Lab name,Action for Site,Study 21,EMEA,Screening (1),Screen Failure,Missing Visits,1.0,8.0,Subject continuing,05NOV2025
freq,2202,19840,56,31,2420,113,1073,2097,2094,NaN,431523,5065,43977,NaN,17188,NaN,431535,422104,NaN,310494,NaN,396871,431418,431418,391,NaN,NaN,127,46,15,15,190,190,26442,25359,8871,18548,5,18046.0,5645.0,286,49
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.583650,NaN,NaN,NaN,4.363465,NaN,1.067742,NaN,NaN,27.999971,NaN,7.943692,NaN,NaN,NaN,NaN,2025-10-15 11:11:11.186440960,30.704420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,27.100000,NaN,1.000000,NaN,NaN,NaN,NaN,2025-04-11 00:00:00,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,2.000000,NaN,1.000000,NaN,NaN,28.000000,NaN,3.000000,NaN,NaN,NaN,NaN,2025-10-06 06:00:00,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,NaN,NaN,NaN,3.000000,NaN,1.000000,NaN,NaN,28.000000,NaN,7.000000,NaN,NaN,NaN,NaN,2025-10-23 00:00:00,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.750000,NaN,NaN,NaN,6.000000,NaN,1.000000,NaN,NaN,28.000000,NaN,12.000000,NaN,NaN,NaN,NaN,2025-11-02 18:00:00,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,981.000000,NaN,NaN,NaN,70.000000,NaN,3.000000,NaN,NaN,28.100000,NaN,198.000000,NaN,NaN,NaN,NaN,2025-11-06 00:00:00,554.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Fix visit_date dtype

inactivated["visit_date"] = pd.to_datetime(
    inactivated["visit_date"],
    errors="coerce"   # invalid dates → NaT
)


/tmp/ipykernel_48918/1262011687.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  inactivated["visit_date"] = pd.to_datetime(


In [30]:
# Inactivated Forms & Loglines
# Subject-level Aggregation

import pandas as pd

# Safety checks
required_cols = [
    "study_id",
    "site_id",
    "subject_id",
    "form_name",
    "folder_name",
    "audit_action",
    "data_on_record",
    "visit_date"
]

missing = [c for c in required_cols if c not in inactivated.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Normalize data_on_record to boolean
inactivated["data_on_record"] = (
    inactivated["data_on_record"]
    .astype(str)
    .str.upper()
    .isin(["Y", "YES", "TRUE", "1"])
)

# Perform aggregation
inactivated_agg = (
    inactivated
    .groupby(["study_id", "site_id", "subject_id"], as_index=False)
    .agg(
        total_inactivated_records=("audit_action", "count"),
        unique_forms_inactivated=("form_name", "nunique"),
        unique_folders_inactivated=("folder_name", "nunique"),
        records_with_data_present=("data_on_record", "sum"),
        records_without_data=("data_on_record", lambda x: (~x).sum()),
        unique_audit_actions=("audit_action", "nunique"),
        earliest_event_date=("visit_date", "min"),
        latest_event_date=("visit_date", "max")
    )
)


In [33]:

inactivated_agg.head()



,study_id,site_id,subject_id,total_inactivated_records,unique_forms_inactivated,unique_folders_inactivated,records_with_data_present,records_without_data,unique_audit_actions,earliest_event_date,latest_event_date
0,Study 17,Site 14,Subject 25,0,0,1,0,4,0,NaT,NaT
1,Study 17,Site 14,Subject 26,0,0,5,0,11,0,NaT,NaT
2,Study 17,Site 15,Subject 39,0,0,9,0,23,0,2025-09-18,2025-11-03
3,Study 17,Site 15,Subject 40,0,0,5,0,25,0,2025-10-23,2025-10-28
4,Study 17,Site 16,Subject 55,0,0,1,0,2,0,NaT,NaT


In [34]:
inactivated_agg.shape

(1303, 11)

In [43]:
inactivated.describe(include="all")


,study_id,country,site_id,subject_id,subject_status_overall,subject_status_form,folder_name,visit_date,form_type,days_page_missing,source_file,form_name,data_on_record,record_position,audit_action,open_issue_count_subject,coding_system,dictionary_name,dictionary_version,form_oid,logline_id,field_oid,coding_status,require_coding,visit_name,actual_date,days_outstanding,lab_category,lab_date,lab_test_name,lab_test_description,lab_issue,comments,project_name,region,latest_visit,subject_status_primary,input_files,cpmd,ssm,subject_status_visit,projected_date
count,3369,124006,3369,3369,3365,228,3369,526,3369,526.000000,555988,22882,555988,11693.000000,66829,310.000000,431535,431535,431418.000000,431535,431535.000000,431535,431535,431535,1339,354,1086.000000,253,253,253,253,253,253,52536,52516,52531,52529,5,52546,52551,340,732
unique,4,113,547,1291,8,6,70,NaN,4,NaN,39,148,2,NaN,8,NaN,1,2,NaN,6,NaN,3,2,2,26,NaN,NaN,3,41,46,46,2,2,6,3,180,8,1,71,378,5,136
top,Study 22,USA,Site 15,Subject 2649,On Trial,Subject continuing,Visit 5 (Month 21),NaN,Visit Level,NaN,Global Coding Report_Medra 4_updated.xlsx,Form 14,False,NaN,Record Inactivated.,NaN,MedDRA Coding Report,MedDRAMedHistory,NaN,MHR002_1,NaN,MHTERM,Coded Term,No,Visit 5 (Month 21),NaN,NaN,CHEMISTRY,28-OCT-2025,APTT,Activated partial\nthromboplastin time,Missing Lab name,Action for Site,Study 21,EMEA,Screening (1),Screen Failure,Missing Visits,1,8,Subject continuing,05NOV2025
freq,2202,19840,56,31,2420,113,1073,NaN,2094,NaN,431523,5065,512011,NaN,17188,NaN,431535,422104,NaN,310494,NaN,396871,431418,431418,391,NaN,NaN,127,46,15,15,190,190,26442,25359,8871,18548,5,18046,5645,286,49
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-12 02:57:56.806083584,NaN,61.583650,NaN,NaN,NaN,4.363465,NaN,1.067742,NaN,NaN,27.999971,NaN,7.943692,NaN,NaN,NaN,NaN,2025-10-15 11:11:11.186440960,30.704420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-09 00:00:00,NaN,1.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,27.100000,NaN,1.000000,NaN,NaN,NaN,NaN,2025-04-11 00:00:00,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-09 06:00:00,NaN,15.000000,NaN,NaN,NaN,2.000000,NaN,1.000000,NaN,NaN,28.000000,NaN,3.000000,NaN,NaN,NaN,NaN,2025-10-06 06:00:00,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-10-20 00:00:00,NaN,23.000000,NaN,NaN,NaN,3.000000,NaN,1.000000,NaN,NaN,28.000000,NaN,7.000000,NaN,NaN,NaN,NaN,2025-10-23 00:00:00,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-10-29 00:00:00,NaN,64.750000,NaN,NaN,NaN,6.000000,NaN,1.000000,NaN,NaN,28.000000,NaN,12.000000,NaN,NaN,NaN,NaN,2025-11-02 18:00:00,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-12 00:00:00,NaN,981.000000,NaN,NaN,NaN,70.000000,NaN,3.000000,NaN,NaN,28.100000,NaN,198.000000,NaN,NaN,NaN,NaN,2025-11-06 00:00:00,554.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,subjects_inactivated,total_inactivated_records
count,4.000000,4.0
mean,325.750000,0.0
std,492.307069,0.0
min,1.000000,0.0
25%,25.000000,0.0
50%,126.000000,0.0
75%,426.750000,0.0
max,1050.000000,0.0
